In [1]:
from value_today import scraper

In [2]:
scraper = scraper.Scraper()

In [3]:
scraper.number_companies()

5083

In [4]:
raw_data = scraper.valuetoday_usa_scraper(20)

In [5]:
valuetoday_df = scraper.df_cleaning(raw_data)

,Company Name,Company Business,Market Value,Value Metric,Annual Revenue,Revenue Metric,Operating Income,Op Income Metric,Net Income,Net Income Metric,Assets,Assets Metric,Liabilities,Liabilities Metric
0,APPLE,Technology,2256,Billion,274.51,Billion,66.28,Billion,57.41,Billion,2020,Million,"245,062",Million
1,MICROSOFT CORPORATION,Technology,1682,Billion,143.00,Billion,52.95,Billion,44.30,Billion,"301,311",Million,"183,007",Million
2,ALPHABET,Technology,1185,Billion,182.52,Billion,41.22,Billion,40.26,Billion,299.243,Billion,86.32,Billion
3,AMAZON.COM,eCommerce,1634,Billion,386.06,Billion,22.89,Billion,21.33,Billion,"110,908",Million,"184,586",Million
4,FACEBOOK,Communication Services,778.04,Billion,85.96,Billion,32.67,Billion,29.15,Billion,"139,691",Million,"29,244",Million
5,BERKSHIRE HATHAWAY,Financial Services,543.68,Billion,327.22,Billion,None,None,81.41,Billion,829.946,Billion,410.73,Billion
6,J P MORGAN CHASE & CO,Financial Services,387.33,Billion,122.90,Billion,None,None,29.10,Billion,"3,386",Billion,"3,106",Billion
7,VISA,Financial Services,482.17,Billion,21.84,Billion,14.09,Billion,10.86,Billion,"77,884",Million,"42,360",Million
8,JOHNSON & JOHNSON,Healthcare,414.31,Billion,82.58,Billion,19.73,Billion,14.71,Billion,"158,380",Million,"95,402",Million
9,WALMART,Retail,407.84,Billion,523.96,Billion,22.54,Billion,14.88,Billion,"237,382",Million,"162,072",Million
